# Deep Learning: Actividad 2: Reinforcement Learning: **Frozen lake problem**

Grupo 8:
- Ricardo Castillo Rodríguez
- Miriam Santana
- Katherine Escobar


# Introducción al problema de FrozenLake ❄️

El entorno FrozenLake es un clásico problema de Reinforcement Learning desarrollado por OpenAI Gym.

Se representa como una cuadrícula de 4x4 donde:
- `S` marca la posición de inicio (Start).
- `F` son celdas seguras (Frozen - Hielo seguro).
- `H` son agujeros peligrosos (Holes) que hacen fallar el episodio.
- `G` es la meta final (Goal).

El objetivo del agente es aprender a desplazarse desde `S` hasta `G`:
- Evitando caer en los agujeros.
- Tomando el menor número de pasos posible.

En esta actividad trabajaremos con la versión **no resbaladiza** (`is_slippery=False`), donde los movimientos son totalmente deterministas (el agente se mueve exactamente en la dirección que elige).

Este entorno nos permitirá entrenar un agente utilizando algoritmos de aprendizaje por refuerzo, concretamente **Q-Learning**, para que descubra la mejor estrategia para alcanzar el objetivo de forma segura y eficiente.

Documentación: https://www.gymlibrary.dev/environments/toy_text/frozen_lake/


# Objetivos, instrucciones y evaluación

Objetivos
- Conseguir movermos aleatoriamente hasta cumplir el objetivo
- Conseguir que el agente aprenda con Q-learning
- (Opcional) Probar con otros hiperparámetros
- (Opcional) Modificar la recompensa

Consideraciones
- No hay penalizaciones
- Si el agente cae en un "hole", entonces done = True y se queda atascado sin poder salir (al igual que ocurre cuando llega al "goal")

Normas a seguir

- Se debe entregar un **ÚNICO GOOGLE COLAB notebook** (archivo .ipynb) que incluya las instrucciones presentes y su **EJECUCIÓN!!!**.
- Poner el nombre del grupo en el nombre del archivo y el nombre de todos los integrantes del grupo al inicio del notebook.

Criterio de evaluación

- Seguimiento de las normas establecidas en la actividad.
- Corrección en el uso de algoritmos, modelos y formas idiomáticas en Python.
- El código debe poder ejecutarse sin modificación alguna en Google Colaboratory.

## **Instalamos librerías**

In [1]:
!pip install numpy==1.24.4
!pip install gym==0.17.3

In [2]:
import gym
import numpy as np
from time import sleep
from IPython.display import clear_output
import random as rd

##**Definición del entorno**

In [8]:
# Definimos el entorno
env = gym.make('FrozenLake-v0', desc=None, map_name="4x4", is_slippery=False)

In [9]:
# Fijamos una semilla
seed_value = 42
env.seed(seed_value)
np.random.seed(seed_value)

import random
random.seed(seed_value)

In [10]:
env.reset() # En este caso, empieza desde la misma posición inicial
print(env.render())


SFFF
FHFH
FFFH
HFFG
None


In [11]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(4)
State Space Discrete(16)


Acciones posibles:
* 0: izquierda
* 1: abajo
* 2: derecha
* 3: arriba

In [12]:
# Identificador de estado
state = env.s
print("State:", state)

State: 0


## **¡Nos movemos aleatoriamente!**

In [13]:
steps = 0
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [14]:
# Acciones: 0=izquierda, 1=abajo, 2=derecha, 3=arriba
action = 1
state, reward, done, info = env.step(action)

print("State:", state)
print(state, reward, done, info)

env.s = state
env.render()

steps += 1

print(f"Step: {steps}")

State: 4
4 0.0 False {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG
Step: 1


In [15]:
# Número de episodios que vamos a jugar
episodes = 10

# Bucle para cada episodio
for episode in range(episodes):
    # Reiniciamos el entorno al inicio de cada episodio
    state = env.reset()
    done = False  # Indicador de si hemos terminado el episodio
    steps = 0     # Contador de pasos realizados

    print(f"Episode {episode+1}")
    sleep(1)


    while not done: # Mientras no hayamos terminado (ni caído en agujero ni alcanzado la meta)
        action = env.action_space.sample() # Elegimos una acción aleatoria (exploración total)

        next_state, reward, done, info = env.step(action) # Aplicamos la acción y recogemos la información resultante

        clear_output(wait=True) # Mostramos visualmente el entorno tras cada movimiento
        env.render()

        state = next_state # Actualizamos el estado actual

        steps += 1 # Aumentamos el contador de pasos

        sleep(0.5) # Pequeña pausa para ver mejor la simulación

    # Mensaje al finalizar el episodio
    if reward == 1:
        print(f"¡Objetivo conseguido en {steps} pasos!")
    else:
        print(f"Caíste en un agujero después de {steps} pasos.")

    sleep(1)

  (Right)
SFFF
FHFH
FFFH
HFFG
Caíste en un agujero después de 2 pasos.


# 1️⃣ **Primer intento**

## **Entrenamiento: Q-Learning**

In [23]:
# Inicializamos la Q-Table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Definimos hiperparámetros
alpha = 0.8    # tasa de aprendizaje
gamma = 0.95   # factor de descuento
epsilon = 0.1  # tasa de exploración (ε-greedy)
episodes = 10000  # número de episodios de entrenamiento

# Entrenamiento
for episode in range(episodes):
    state = env.reset()
    done = False

    while not done:
        # Política ε-greedy: explora o explota
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Exploración
        else:
            action = np.argmax(q_table[state])  # Explotación

        # Ejecutamos acción
        next_state, reward, done, info = env.step(action)

        # Penalización por paso para que entienda que cuantos menos pasos mejor y para que no vuelva sobre los pasos ya dados
        reward = reward - 0.01

        # Actualizamos Q-Table usando la fórmula Q-Learning
        q_table[state, action] = q_table[state, action] + alpha * (
            reward + gamma * np.max(q_table[next_state]) - q_table[state, action]
        )

        # Actualizamos estado
        state = next_state

## **Evaluación: Resultados de Q-Learning inicial**

In [20]:
test_episodes = 10
successes = 0

for episode in range(test_episodes):
    state = env.reset()
    done = False
    steps = 0

    print(f"Episode {episode+1}")
    sleep(1)

    while not done:
        action = np.argmax(q_table[state])
        next_state, reward, done, info = env.step(action)

        clear_output(wait=True)
        env.render()

        state = next_state
        steps += 1

        sleep(0.5)

    if reward == 1:
        print(f"¡Objetivo conseguido en {steps} pasos!")
        successes += 1
    else:
        print(f"Caíste en un agujero después de {steps} pasos.")

    sleep(1)

# Mostramos porcentaje de éxito
print(f"\nPorcentaje de éxito: {successes/test_episodes*100:.2f}%")

  (Right)
SFFF
FHFH
FFFH
HFFG
Caíste en un agujero después de 2 pasos.

Porcentaje de éxito: 0.00%


Observamos que el agente no consigue llegar al objetivo.

## **Análisis**

Dado que FrozenLake es un entorno con alta dificultad por la disposición de los agujeros, decidimos ajustar los hiperparámetros para mejorar el aprendizaje del agente:

Acciones tomadas:

- Incrementar el epsilon para fomentar más exploración.
- Aumentar el número de episodios.
- Reducir ligeramente la penalización por paso.

Con los nuevos hiperparámetros, esperamos que el agente explore más, descubra caminos exitosos y reduzca el número de caídas en agujeros.



# 2️⃣ **Segundo intento**

## **Entrenamiento**

In [21]:
# Inicializamos la Q-Table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Definimos hiperparámetros
alpha = 0.8    # tasa de aprendizaje
gamma = 0.95   # factor de descuento
epsilon = 0.5  # tasa de exploración (ε-greedy)
episodes = 30000  # número de episodios de entrenamiento

# Entrenamiento
for episode in range(episodes):
    state = env.reset()
    done = False

    while not done:
        # Política ε-greedy: explora o explota
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Exploración
        else:
            action = np.argmax(q_table[state])  # Explotación

        # Ejecutamos acción
        next_state, reward, done, info = env.step(action)

        # Penalización por paso
        reward = reward - 0.001

        # Actualizamos Q-Table usando la fórmula Q-Learning
        q_table[state, action] = q_table[state, action] + alpha * (
            reward + gamma * np.max(q_table[next_state]) - q_table[state, action]
        )

        # Actualizamos estado
        state = next_state

## **Evaluación**

In [22]:
test_episodes = 10
successes = 0

for episode in range(test_episodes):
    state = env.reset()
    done = False
    steps = 0

    print(f"Episode {episode+1}")
    sleep(1)

    while not done:
        action = np.argmax(q_table[state])
        next_state, reward, done, info = env.step(action)

        clear_output(wait=True)
        env.render()

        state = next_state
        steps += 1

        sleep(0.5)

    if reward == 1:
        print(f"¡Objetivo conseguido en {steps} pasos!")
        successes += 1
    else:
        print(f"Caíste en un agujero después de {steps} pasos.")

    sleep(1)

# Mostramos porcentaje de éxito
print(f"\nPorcentaje de éxito: {successes/test_episodes*100:.2f}%")

  (Right)
SFFF
FHFH
FFFH
HFFG
¡Objetivo conseguido en 6 pasos!

Porcentaje de éxito: 100.00%


Después de entrenar con la nueva configuración (mayor `epsilon`, más episodios y penalización de paso reducida), evaluamos el agente.

Resultados observados:
- ¡Objetivo conseguido en todos los episodios!
- El agente logra encontrar el camino al objetivo sin caer en agujeros.

**Porcentaje de éxito: 100%**

Esto demuestra una clara mejora en el desempeño del agente respecto al primer intento, confirmando que los ajustes de hiperparámetros fueron acertados.


# **Conclusión final**

El proceso de iteración, análisis de errores y ajuste de hiperparámetros permitió que el agente aprendiera a resolver el entorno FrozenLake de manera eficiente.

Esto resalta la importancia de:
- Fomentar suficiente exploración durante el entrenamiento.
- Permitir que el agente aprenda de sus errores a lo largo de más episodios.
- Ajustar correctamente las penalizaciones para guiar el comportamiento deseado.

En definitiva, el agente fue capaz de alcanzar el objetivo en el 100% de las pruebas tras la mejora, reflejando un aprendizaje exitoso.